In [1]:
from RGBD_XYZ.rgb_xy import *
from RGBD_XYZ.xyd_xyz import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
rgb2xy = rgb_xy()
xyd2xyz = xyd_xyz()
def img2xyz(rgb, dep):
    xy = rgb2xy.detect(rgb)
    x = xy[0].astype(np.int32)
    y = xy[1].astype(np.int32)
    # get xy for image
    xyd = np.concatenate((xy, np.array(dep[x][y][:1])), axis=0)
    # get depth for dep
#     plt.imshow(rgb)
#     print(xyd)
    xyz = xyd2xyz.detect(xyd)
    # get the xyz in the space
#     print(xyz)
    return xyz

MyLoss


In [3]:
# ----- test ----
rgb, dep = read_img_dep("[0.5513610201719624, 0.1887571504509108, 0.75]")
# ----- test ----
img2xyz(rgb, dep)

d:\annaconda\envs\pytorch\lib\site-packages\torch\nn\modules\upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
d:\annaconda\envs\pytorch\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


array([[0.5301885, 0.2018118, 0.7484169]], dtype=float32)

In [4]:
import os
import numpy as np
file_name = []
for root, dirs, files in os.walk("D:/images/images/rgb/", topdown=False):
        file_name = files
print(file_name[np.random.randint(1, 99999)][:-4])

[0.7411835697053846, -0.2628055284125354, 0.75]


In [8]:
from DDPG_SERVER.ddpgmain import *
robot = Robot()
rgb, dep = read_img_dep(file_name[np.random.randint(1, 99999)][:-4])
position = img2xyz(rgb, dep)[0]
robot.Box_position = position
# robot.random_box_position()
Box_position = robot.read_box_position()
# Box_position = [-0.04604054586892892, 0.08798244770703123, -0.11748548030476458]
rl = DDPG()
rl.load_model()
print(Box_position)
var = 0.0
total_rewards = []
step_sums = []
# 主循环
for i in range(1, MAX_EPISODES):
#     robot.random_box_position()
    rgb, dep = read_img_dep(file_name[np.random.randint(1, 99999)][:-4])
    robot.Box_position = img2xyz(rgb, dep)[0]
    Box_position = robot.read_box_position()
    print("Box_Position:", Box_position)
    recent_end_goal = [0.715976, 0.029221, 1.0]
    robot.end_goal = recent_end_goal    # 末端坐标位置
    if i % 50 == 0:
        print("\n------------------Episode:{0}------------------".format(i))
    st = 0
    rw = 0
    # print "cube position:", Box_position
    # 存储夹爪距离木块的距离
    now_dis = math.sqrt(math.pow(recent_end_goal[0] - Box_position[0], 2)
                        + math.pow(recent_end_goal[1] - Box_position[1], 2)
                        + math.pow(recent_end_goal[2] - Box_position[2], 2))
    robot.dis = now_dis
    # 读取end_goal
    state = robot.get_state()
    if i % 500 == 0:
        print("****************memory counter:{0}****************".format(rl.memory_counter))
    end = time.clock()
    begin = time.clock()
    # 分成末端坐标和 rgbd
    while True:
        st += 1
        # ------------ choose action ------------
        s = np.array(state).reshape(1, -1)
        goal = np.array(Box_position).reshape(1, -1)
        x = np.concatenate((s, goal), axis=1)
        action = rl.choose_action(x)
        if st < 5:
            print("action: ", action)
        # ------------ -------------- ------------
        next_state, r, done, success = robot.test_step(action, var)

        # ----------- store transition -----------
        s_next = np.array(next_state).reshape(1, -1)
        x1 = np.concatenate((s_next, goal), axis=1)
        rl.store_transition(x, action, r, x1)
        # ----------- ---------------- -----------
        if rl.memory_counter > MEMORY_CAPACITY:
            var *= .9995
            rl.learn()
            if st == 1:
                print(".....................learn.....................")
        rw += r
        state = next_state
        if done or st >= MAX_EP_STEPS:
            # if rl.memory_counter < 10000 and i % 50 == 0:
            #     break
            print("Step:{0}, total reward:{1}, average reward:{2}, {3}".format(st, rw, rw*1.0/st, "sucess"if success else "-----"))
            total_rewards.append(rw)
            step_sums.append(st)
            break
    print("{0}\n".format(state))
#     if rl.memory_counter > MEMORY_CAPACITY and i % 50 == 0:
#         print("save model!")
#         rl.save_model()
file_ob = open('table.txt', 'w')
for i in total_rewards:
    file_ob.write(str(i))
    file_ob.write('\n')
file_ob.close()

file_ob = open('step.txt', 'w')
for i in step_sums:
    file_ob.write(str(i))
    file_ob.write('\n')
file_ob.close()

ModuleNotFoundError: No module named 'fetch_moveit_config'